In [1]:
import cv2
from skimage.filters import threshold_otsu
import numpy as np
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from skimage.draw import rectangle
%matplotlib inline 

In [192]:
im_in = cv2.imread("er4.png", cv2.IMREAD_GRAYSCALE)
th, im_th = cv2.threshold(im_in, 220, 255, cv2.THRESH_BINARY_INV)
###################Remove words###############
structureElm =  np.array((
	[1,1,1],
	[1,1,1]), dtype="int")
closed = closing(im_th, structureElm)
for i in range(20):
    closed = closing(closed, structureElm)
cv2.imwrite("closed.png", closed)

im2, contours_closed, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [193]:
# Mask used to flood filling.
# Notice the size needs to be 2 pixels than the image.
hImg, wImg = im_th.shape[:2]
area = hImg*wImg
mask = np.zeros((hImg+2, wImg+2), np.uint8)

im_floodfill = im_th.copy()
##################Flood fill corners#####
cv2.floodFill(im_floodfill, mask, (0,0), 255)
cv2.floodFill(im_floodfill, mask, (wImg-1,0), 255)
cv2.floodFill(im_floodfill, mask, (0,hImg-1), 255)
cv2.floodFill(im_floodfill, mask, (wImg-1,hImg-1), 255)

print("corners Filled")

corners Filled


In [194]:
######################cv contours#########
# Invert floodfilled image
im_floodfill_inv = cv2.bitwise_not(im_floodfill)
im2, contours_cv, hierarchy = cv2.findContours(im_floodfill_inv, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
im_empty = np.ones((hImg, wImg,3), np.uint8) * 255
cv2.drawContours(im_empty, contours_cv, -1, (0,0,0), 1)
cv2.imwrite("contoured.png", im_empty)
#########################################
# Display images.
cv2.imwrite("Thresholded_Image.png", im_th)
cv2.imwrite("Floodfilled_Image.png", im_floodfill)
cv2.imwrite("Inverted_Floodfilled_Image.png", im_floodfill_inv)
#start flooding from corners?

True

In [195]:
def isValid(imV,m, n, i, j):
    #print(i<0 , i>= m , j<0 , j>= n , imV[j][i] == 0)
    #print(i,j,m,n)
    if (i<0 or i>= m or j<0 or j>= n or imV[j][i] == 0):
        return False
    return True
 
# FloodFill function
def floodFillBFS(screen,m, n, xImg, yImg):
    queue = []
     
    # Append the position of starting
    # pixel of the component
    queue.append([xImg, yImg])
 
    # Color the pixel with the new color
    # if(screen[yImg][xImg]==0):
    #     #print("hi")
    #     return
    screen[yImg][xImg] = 0
 
    # While the queue is not empty i.e. the
    # whole component having prevC color
    # is not colored with newC color
    while queue:
        # Dequeue the front node
        currPixel = queue.pop() 
        posX = currPixel[0]
        posY = currPixel[1]
        # Check if the adjacent
        # pixels are valid
        if isValid(screen, m, n,posX + 1, posY):
           # print("valid r")
            screen[posY][posX + 1] = 0
            queue.append([posX + 1, posY])
         
        if isValid(screen, m, n,posX-1, posY):
           # print("valid l")
            screen[posY][posX-1]= 0
            queue.append([posX-1, posY])
         
        if isValid(screen, m, n, posX, posY + 1):
           # print("valid d")
            screen[posY + 1][posX]= 0
            queue.append([posX, posY + 1])
         
        if isValid(screen, m, n,posX, posY-1):
            #print("valid u")
            screen[posY-1][posX]= 0
            queue.append([posX, posY-1])

In [196]:
im_empty_2D=cv2.cvtColor(im_empty,cv2.COLOR_BGR2GRAY) 
for cnt in contours_closed:
    x,y,w,h = cv2.boundingRect(cnt)
    if w>20 and w<550 and h>5 and h<20:
        i = int((2*x+w)/2) 
        i = i if i>0 else 0
        j = y-1 if y>0 else 0
        im_empty[j][i] = [0,0,255]
        im_empty[y][x] = [0,255,0]
        im_empty[y][x+w] = [0,255,0]
        im_empty[y+h][x] = [0,255,0]
        im_empty[y+h][x+w] = [0,255,0]
        floodFillBFS(im_empty_2D,wImg,hImg, i,j)

cv2.imwrite("contoured.png", im_empty)


True

In [197]:
structureElm =  np.array((
	[1,1,1],
	[1,1,1],
    [1,1,1]), dtype="int")
im_empty_2D = dilation(im_empty_2D, structureElm)

cv2.imwrite("flooded.png", im_empty_2D)


True